In this notebook I will look to fetch the weekly GGR totals across domain and sports.

### Imports

In [3]:
import os
os.chdir("../..")

In [5]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from src.bigquery.functions import bq_query

### LiveSCore group bets

In [6]:
startDate = "2023-04-01"
endDate = "2023-04-27"

In [7]:
bets_query = f"""
-- Set timeframe
DECLARE startDate date;
DECLARE endDate date;
SET startDate = "{startDate}";
SET endDate = "{endDate}";

-- Bets from 1st April (all brands ie VB and LS)

WITH vb_bets AS ( -- Grab all VB bets from 1st April
  SELECT
    betID
    ,purchaseBetID
    ,relatedToBetID
    ,vb_betsnap.userID
    ,isTest
    ,CASE
      WHEN vb_betsnap.domain LIKE 'VirginBet%' THEN 'VB'
      ELSE 'NOT VB'
    END as domain
    ,provider
    ,vb_betsnap.lastSettlementDateTime as lastSettlementDateTime
    ,lastSettlementDate
    ,DATE_SUB(lastSettlementDate, INTERVAL CAST(FORMAT_DATE('%u', lastSettlementDate) AS INT) -1 DAY) as week_commencing
    ,betType
    ,betSubType
    ,betGroup
    ,betCategory
    ,s.event.sport as Sport -- NB betSport is not used as this includes Mixed Sports which obviously effects dividing GGR per sport
    ,betCount
    ,betOddsPlaced
    ,betOddsApplied
    ,s.relGGR.GBP AS rel_GGR_GBP --NB Not using 'GGR.GBP as GGR_GBP' due to acca's
  FROM `gamesys-eu-live-sbtech-data.sb_tech_presentation.BetSnapshot` as vb_betsnap
  CROSS JOIN UNNEST(selections) AS s
  LEFT JOIN `gamesys-eu-live-sbtech-data.sb_tech_presentation.User` as vb_users ON vb_betsnap.userID = vb_users.userID
  WHERE 1=1
    AND
    lastSettlementDate >= startDate
    AND
    lastSettlementDate <= endDate
    AND
    vb_betsnap.domain LIKE 'VirginBet%'
    AND
    isTest IS FALSE

)

, ls_bets AS ( -- Grab all LS bets from 1st April
  SELECT
    betID
    ,purchaseBetID
    ,relatedToBetID
    ,ls_betsnap.userID
    ,isTest
    ,ls_betsnap.domain
    ,provider
    ,ls_betsnap.lastSettlementDateTime as lastSettlementDateTime
    ,lastSettlementDate
    ,DATE_SUB(lastSettlementDate, INTERVAL CAST(FORMAT_DATE('%u', lastSettlementDate) AS INT) -1 DAY) as week_commencing
    ,betType
    ,betSubType
    ,betGroup
    ,betCategory
    ,s.event.sport as Sport -- NB betSport is not used as this includes Mixed Sports which obviously effects dividing GGR per sport,betSport
    ,betCount
    ,betOddsPlaced
    ,betOddsApplied
    ,s.relGGR.GBP AS rel_GGR_GBP --NB Not using 'GGR.GBP as GGR_GBP' due to acca's
  FROM `ls-africa-data-eu-live.lithium_warehouse.BetSnapshot` AS ls_betsnap
  CROSS JOIN UNNEST(selections) AS s
  LEFT JOIN `ls-africa-data-eu-live.lithium_warehouse.User` AS ls_users ON ls_betsnap.userID = ls_users.userID
  WHERE 1=1
    AND
    lastSettlementDate >= startDate
    AND
    lastSettlementDate <= endDate
    AND
    ls_betsnap.domain LIKE 'LiveScore%'
    AND
    isTest IS FALSE
)

, lsg_bets AS ( -- lsg - LiveScore Group
  SELECT 
    *
  FROM vb_bets

  UNION ALL

  SELECT 
    *
  FROM ls_bets

)

SELECT * FROM lsg_bets
"""

In [8]:
bets_df = bq_query(bets_query)

c:\Users\james.lai\OneDrive - AnzoGroup\Documents\code\personal_repo\.venv\Lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Format data to weekly summary

In [29]:
weekly_summary_df = (
    bets_df
    .groupby(['week_commencing','domain','Sport'])
    .agg({'rel_GGR_GBP':['sum']})
    .reset_index()
)
weekly_summary_df.columns = ['week_commencing', 'domain', 'sport', 'total_GGR']

weekly_summary_df = weekly_summary_df.sort_values(
    by = ['week_commencing', 'total_GGR', 'sport'],
    ascending = [True, False, True]
)

### Dataframe filter

In [24]:
import numpy as np
weekly_summary_df.week_commencing.unique().tolist()

[datetime.date(2023, 3, 27),
 datetime.date(2023, 4, 3),
 datetime.date(2023, 4, 10),
 datetime.date(2023, 4, 17),
 datetime.date(2023, 4, 24)]

In [25]:
ALL = 'ALL' # All option used so that all filters can be removed

# function get year list
def filter_list(input_list):
    filter_list = ['ALL'] + input_list
    return filter_list

# Now we will initialise the dropdown:
dropdown_weeks = widgets.Dropdown(options = filter_list(weekly_summary_df.week_commencing.unique().tolist()))

# create a new instance of Output
output_week = widgets.Output()

def dropdown_weeks_eventhandler(change):
    output_week.clear_output()
    with output_week:
        if (change.new == ALL):
            display(weekly_summary_df)
        else:
            display(weekly_summary_df.loc[lambda dfr_: dfr_.week_commencing == change.new])

# We will then bind the handler to the dropdown:
dropdown_weeks.observe( dropdown_weeks_eventhandler, names='value')

display(dropdown_weeks)

Dropdown(options=('ALL', datetime.date(2023, 3, 27), datetime.date(2023, 4, 3), datetime.date(2023, 4, 10), da…

In [28]:
display(output_week)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '    week_commencing          domain   …